In [51]:
from py2neo import Graph
from bs4 import BeautifulSoup
import pandas as pd
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
import wikipedia
import spacy
nlp = spacy.load("en_core_web_sm")
import gensim
from collections import defaultdict
from gensim.parsing.preprocessing import strip_multiple_whitespaces, preprocess_string, remove_stopwords, strip_tags, strip_punctuation 

In [2]:
graph = Graph(password="kshen3778")

In [3]:
graph.run("MATCH (n:Category) RETURN n LIMIT 25").data()

[{'n': (_5214855:Category:Page {id: 690070, isNew: false, isRedirect: false, title: 'Futurama'})},
 {'n': (_5214856:Category:Page {id: 690451, isNew: false, isRedirect: false, title: 'World_War_II'})},
 {'n': (_5214857:Category:Page {id: 690571, isNew: false, isRedirect: false, title: 'Programming_languages'})},
 {'n': (_5214858:Category:Page {id: 690578, isNew: false, isRedirect: false, title: 'Professional_wrestling'})},
 {'n': (_5214859:Category:Page {id: 690637, isNew: false, isRedirect: false, title: 'Algebra'})},
 {'n': (_5214860:Category:Page {id: 690649, isNew: false, isRedirect: false, title: 'Anime'})},
 {'n': (_5214861:Category:Page {id: 690672, isNew: false, isRedirect: false, title: 'Abstract_algebra'})},
 {'n': (_5214862:Category:Page {id: 690747, isNew: false, isRedirect: false, title: 'Mathematics'})},
 {'n': (_5214863:Category:Page {id: 690777, isNew: false, isRedirect: false, title: 'Linear_algebra'})},
 {'n': (_5214864:Category:Page {id: 690803, isNew: false, isRedir

In [20]:
graph.run("MATCH (n:Category { title: 'Cold_War' }) RETURN n").data()

[{'n': (_5214918:Category:Page {id: 691614, isNew: false, isRedirect: false, title: 'Cold_War'})}]

In [55]:
#get all sub cat
cats = graph.run("MATCH (n:Category{title:'Constraint_logic_programming'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()

In [56]:
len(cats)

2

In [57]:
cats

[{'p': (_6124108:Category:Page {id: 22968506, isNew: false, isRedirect: false, title: 'Constraint_programming'})},
 {'p': (_5594005:Category:Page {id: 1188828, isNew: false, isRedirect: false, title: 'Logic_programming'})}]

In [27]:
getBadCategories("Artificial_intelligence")

['Areas_of_computer_science',
 'Cognitive_science',
 'Computational_neuroscience',
 'Cybernetics',
 'Emerging_technologies',
 'Formal_sciences',
 'Futurology',
 'Intelligence_by_type',
 'Personhood',
 'Technology_in_society',
 'Unsolved_problems_in_computer_science']

In [63]:
#check if a specific sub category references the select category an two way relationship (which we don't want)
subs = graph.run("MATCH (n:Category{title:'Emerging_technologies'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()

In [64]:
subs

[]

In [20]:
#verify if category actually exists on online Wikipedia
def categoryExist(name):
    res = requests.get("https://en.wikipedia.org/wiki/Category:" + name)
    soup = BeautifulSoup(res.content, "html.parser")
    div = soup.find("div", {"id": "mw-normal-catlinks"})
    if(div):
        return True
    else:
        return False

#Use beautifulsoup to scrape the bottom categories of a parent category (these we don't want)
def getBadCategories(parent):
    res = requests.get("https://en.wikipedia.org/wiki/Category:" + parent)
    soup = BeautifulSoup(res.content, "html.parser")
    div = soup.find("div", {"id": "mw-normal-catlinks"})
    if(div):
        to_return = []
        for item in div.findAll("li"):
            to_return.append("_".join(item.get_text().split(" ")))
        return to_return
    else:
        #parent does not exist
        return []

#get all immediate Wikipedia subcategories of main minus two way connections
def getAllSubCat(parent):
    all_cats = graph.run("MATCH (n:Category{title:'" + parent + "'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()
    bad_cats = getBadCategories(parent)
    filtered = []
    for category in all_cats:
        cat_title = category['p']["title"]
        if (not (cat_title in bad_cats)):
            filtered.append(cat_title)
    #search all these categories to see if any of them match the other way (if so, we remove)
#     filtered = []
#     for category in all_cats:
#         cat_title = category['p']["title"]
#         subs = graph.run("MATCH (n:Category{title:'" + cat_title + "'})-[:BELONGS_TO*..1]-(p:Category) RETURN p LIMIT 100").data()
#         found = False
#         for category_2 in subs:
#             sub_title = category_2['p']["title"]
#             if(sub_title == parent):
#                 found = True
#         if (not found):
#             filtered.append(cat_title)
    
    return filtered

In [77]:
getAllSubCat("Machine_learning")
#getBadCategories("Constraint_logic_programming")

['Artificial_neural_networks',
 'Applied_machine_learning',
 'Unsupervised_learning',
 'Structured_prediction',
 'Loss_functions',
 'Support_vector_machines',
 'Latent_variable_models',
 'Cluster_analysis',
 'Signal_processing_conferences',
 'Artificial_intelligence_conferences',
 'Data_mining_and_machine_learning_software',
 'Machine_learning_algorithms',
 'Bayesian_networks',
 'Machine_learning_portal',
 'Statistical_natural_language_processing',
 'Markov_models',
 'Evolutionary_algorithms',
 'Genetic_programming',
 'Datasets_in_machine_learning',
 'Dimension_reduction',
 'Machine_learning_researchers',
 'Supervised_learning',
 'Semisupervised_learning',
 'Computational_learning_theory',
 'Kernel_methods_for_machine_learning',
 'Learning_in_computer_vision',
 'Inductive_logic_programming',
 'Log-linear_models',
 'Classification_algorithms',
 'Machine_learning_task',
 'Ensemble_learning']

In [79]:
graph.run("MATCH path = (start:Category{title:'Machine_learning'})-[*]-(end:Category{title:'Court_titles'}) RETURN path LIMIT 3").data()

KeyboardInterrupt: 

In [21]:
all_nodes = []
all_relations = [] #child parent
def buildGraph(root, path):
    path = path + "/" + root
    print("Current Path: " + path)
    if(not categoryExist(root)):
        print("Category DNE")
        return
    #add the node
    all_nodes.append(root)
    
    #get all subcategories
    sub_cat = getAllSubCat(root)
    if(len(sub_cat) == 0): #base case
        return
    
    #add all the relationships
    for item in sub_cat:
        all_relations.append(item + " " + root)
   
    #recurse on all sub nodes
    for item in sub_cat:
        buildGraph(item, path)


In [22]:
buildGraph("Machine_learning", "")

Current Path: /Machine_learning
Current Path: /Machine_learning/Artificial_neural_networks
Current Path: /Machine_learning/Artificial_neural_networks/Neural_network_software
Current Path: /Machine_learning/Artificial_neural_networks/Deep_learning
Current Path: /Machine_learning/Applied_machine_learning
Current Path: /Machine_learning/Applied_machine_learning/AlphaGo
Current Path: /Machine_learning/Unsupervised_learning
Current Path: /Machine_learning/Structured_prediction
Current Path: /Machine_learning/Structured_prediction/Graphical_models
Current Path: /Machine_learning/Structured_prediction/Graphical_models/Bayesian_networks
Current Path: /Machine_learning/Structured_prediction/Graphical_models/Causal_inference
Current Path: /Machine_learning/Structured_prediction/Graphical_models/Markov_networks
Current Path: /Machine_learning/Loss_functions
Current Path: /Machine_learning/Support_vector_machines
Current Path: /Machine_learning/Latent_variable_models
Current Path: /Machine_learnin

In [28]:
all_nodes

['Machine_learning',
 'Artificial_neural_networks',
 'Neural_network_software',
 'Deep_learning',
 'Applied_machine_learning',
 'AlphaGo',
 'Unsupervised_learning',
 'Structured_prediction',
 'Graphical_models',
 'Bayesian_networks',
 'Causal_inference',
 'Markov_networks',
 'Loss_functions',
 'Support_vector_machines',
 'Latent_variable_models',
 'Structural_equation_models',
 'Factor_analysis',
 'Cluster_analysis',
 'Clustering_criteria',
 'Cluster_analysis_algorithms',
 'Signal_processing_conferences',
 'Artificial_intelligence_conferences',
 'Data_mining_and_machine_learning_software',
 'Social_network_analysis_software',
 'Machine_learning_algorithms',
 'Genetic_algorithms',
 'Artificial_immune_systems',
 'Gene_expression_programming',
 'Bayesian_networks',
 'Statistical_natural_language_processing',
 'Language_modeling',
 'Markov_models',
 'Hidden_Markov_models',
 'Markov_networks',
 'Evolutionary_algorithms',
 'Genetic_programming',
 'Genetic_algorithms',
 'Artificial_immune_sys

In [27]:
all_relations

['Artificial_neural_networks Machine_learning',
 'Applied_machine_learning Machine_learning',
 'Unsupervised_learning Machine_learning',
 'Structured_prediction Machine_learning',
 'Loss_functions Machine_learning',
 'Support_vector_machines Machine_learning',
 'Latent_variable_models Machine_learning',
 'Cluster_analysis Machine_learning',
 'Signal_processing_conferences Machine_learning',
 'Artificial_intelligence_conferences Machine_learning',
 'Data_mining_and_machine_learning_software Machine_learning',
 'Machine_learning_algorithms Machine_learning',
 'Bayesian_networks Machine_learning',
 'Machine_learning_portal Machine_learning',
 'Statistical_natural_language_processing Machine_learning',
 'Markov_models Machine_learning',
 'Evolutionary_algorithms Machine_learning',
 'Genetic_programming Machine_learning',
 'Datasets_in_machine_learning Machine_learning',
 'Dimension_reduction Machine_learning',
 'Machine_learning_researchers Machine_learning',
 'Supervised_learning Machine_

In [29]:
#save graph to files
with open('nodes.txt', 'w') as f:
    for item in all_nodes:
        f.write("%s\n" % item)

with open('relations.txt', 'w') as f:
    for item in all_relations:
        f.write("%s\n" % item)        

In [40]:
def preprocess(x):
    x = gensim.utils.simple_preprocess(x)
    x = " ".join(x)
    
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, remove_stopwords]
    x = preprocess_string(x, CUSTOM_FILTERS)
    return x

In [48]:
#Extract wikipedia articles (to convert into feature vectors)
file = open("nodes.txt", "r")
articles_cleaned = []
freq_threshold = 3
for line in file:
    print("=====" + line + "=======")
    page = wikipedia.page(wikipedia.search(line.rstrip("\n"))[0])
    content = page.content
    cleaned = preprocess(content) #strip punctuations and stopwords and weird characters
    cleaned = " ".join(cleaned)
    doc = nlp(cleaned)
    lemm = [token.lemma_ for token in doc]
    d = defaultdict(int)
    for item in lemm:
        d[item] += 1
    tokens=[key for key,value in d.items() if value>freq_threshold] #all words with frequency of more than some number
    texts = [word for word in lemm if word in tokens]
    texts = " ".join(texts)
    articles_cleaned.append(texts)
    print(texts)
file.close()

=====Machine_learning
machine learn study algorithm statistical model system use perform specific task pattern inference instead subset artificial intelligence machine learn algorithm build mathematical model base sample datum know training data order prediction decision program perform task machine learn algorithm variety application email vision algorithm perform task machine learn related computational statistic focus make prediction study mathematical optimization method theory application field machine learning datum mining field study machine learning focus datum analysis unsupervised learning application problem machine learn predictive machine learning provide definition algorithm study machine learning field program learn experience class task performance measure performance task measure improve experience definition task machine learn definition define field term compute intelligence machine machine characteristic machine machine learn task machine learn task category supervi

artificial neural network ann system computing system biological neural network brain system learn perform task consider example task example image recognition learn identify image cat example image cat cat result identify cat image prior cat example cat identify example process ann base connect unit node call artificial neuron model neuron biological brain connection biological brain signal neuron artificial neuron receive signal process signal neuron connect ann signal connection real number output neuron compute non function input connection call neuron typically weight adjust learn weight increase signal connection neuron threshold signal signal threshold typically neuron layer layer perform input signal layer input layer layer output layer layer multiple time goal ann approach problem human brain time perform task anns task include speech recognition machine network game consider human create model neural network create learn base neural know learn machine call simulate network cr

neural network software research develop artificial neural network software biological neural network adaptive artificial learn simulator neural network simulator software application artificial biological neural network type neural network produce neural network software simulator simulator neural network research simulator type neural network software research neural network type software neural network artificial neural network simulator component base development environment research platform artificial neural network simulator include neural network simulator neural biological neural network software simulator biological neural biological network simulator include datum analysis simulator research simulator datum analysis simulator application artificial neural network datum mining datum analysis simulator development environment datum analysis simulator use simple neural network datum analysis simulator use network type software use neural datum analysis simulator simulator neura

machine learn study algorithm statistical model system use perform specific task pattern inference instead subset artificial intelligence machine learn algorithm build mathematical model base sample datum know training data order prediction decision program perform task machine learn algorithm variety application email vision algorithm perform task machine learn related computational statistic focus make prediction study mathematical optimization method theory application field machine learning datum mining field study machine learning focus datum analysis unsupervised learning application problem machine learn predictive machine learning provide definition algorithm study machine learning field program learn experience class task performance measure performance task measure improve experience definition task machine learn definition define field term compute intelligence machine machine characteristic machine machine learn task machine learn task category supervise learn algorithm bui

alphago program play board game deepmind google alphago powerful successor call alphago master alphago zero alphazero october alphago program beat human professional player handicap board beat lee sedol game match time program beat dan professional handicap lose lee sedol fourth game lee game game alphago victory alphago award dan korea association lead match lee sedol title alphago year december future summit successor alphago master beat ke jie world rank player time game match powerful alphago zero alphago award professional dan chinese association alphago successor use tree search algorithm move learn machine learn artificial neural network deep learning method human play neural network train alphago game neural improve strength tree search strong play match alphago ke jie deepmind alphago ai research alphago zero achieve victory champion alphago successor alphazero world player chess computer win game chess make use ai method tree search deep beat world chess champion match strong

...

machine learn ensemble method multiple learn algorithm well performance learn algorithm statistical ensemble statistical machine learn ensemble set model typically learn algorithm perform hypothesis space hypothesis good prediction problem hypothesis space hypothesis problem good ensemble combine multiple hypothesis well hypothesis ensemble method multiple hypothesis base multiple classifier system cover hypothesis base prediction ensemble typically prediction single model ensemble learning algorithm perform algorithm decision tree ensemble method example random forest algorithm ensemble technique ensemble technique learning example detection ensemble ensemble learn algorithm train prediction train ensemble represent single hypothesis hypothesis hypothesis space model ensemble show represent training datum single model ensemble technique bag tend reduce problem training datum ensemble tend yield well result model ensemble method seek promote model combine random algorithm like random d

neural network software research develop artificial neural network software biological neural network adaptive artificial learn simulator neural network simulator software application artificial biological neural network type neural network produce neural network software simulator simulator neural network research simulator type neural network software research neural network type software neural network artificial neural network simulator component base development environment research platform artificial neural network simulator include neural network simulator neural biological neural network software simulator biological neural biological network simulator include datum analysis simulator research simulator datum analysis simulator application artificial neural network datum mining datum analysis simulator development environment datum analysis simulator use simple neural network datum analysis simulator use network type software use neural datum analysis simulator simulator neura

decision tree decision tool tree model decision outcome utility decision tree commonly operation research decision analysis tool decision tree node represent represent outcome node represent decision path represent rule decision analysis decision tree influence diagram decision tool value utility decision tree node decision node represent node represent node represent tree commonly operation research operation decision decision tree model model model decision tree decision tree influence diagram utility decision analysis tool example operation research decision tree decision tree draw decision tree node path node path draw decision tree example decision rule decision tree decision rule outcome node condition path rule condition condition condition outcome decision rule rule decision tree commonly decision tree draw analysis example analysis decision utility example model example commonly operation research lifeguard beach beach example example beach lifeguard beach beach lifeguard beac

machine learn ensemble method multiple learn algorithm well performance learn algorithm statistical ensemble statistical machine learn ensemble set model typically learn algorithm perform hypothesis space hypothesis good prediction problem hypothesis space hypothesis problem good ensemble combine multiple hypothesis well hypothesis ensemble method multiple hypothesis base multiple classifier system cover hypothesis base prediction ensemble typically prediction single model ensemble learning algorithm perform algorithm decision tree ensemble method example random forest algorithm ensemble technique ensemble technique learning example detection ensemble ensemble learn algorithm train prediction train ensemble represent single hypothesis hypothesis hypothesis space model ensemble show represent training datum single model ensemble technique bag tend reduce problem training datum ensemble tend yield well result model ensemble method seek promote model combine random algorithm like random d

NameError: name 'f' is not defined

In [50]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(articles_cleaned)
print(vectorizer.get_feature_names())
print(vectors.shape)

['aaai', 'ab', 'ability', 'able', 'abstract', 'access', 'account', 'accuracy', 'achieve', 'acm', 'acquire', 'acquisition', 'action', 'activation', 'activity', 'actual', 'acyclic', 'adaptation', 'adaptive', 'add', 'addition', 'address', 'adjacent', 'adjust', 'advertising', 'ai', 'aic', 'aid', 'aim', 'ais', 'al', 'algorithm', 'algorithmic', 'aliase', 'alice', 'align', 'allow', 'alpha', 'alphago', 'alphazero', 'analysis', 'analyze', 'ann', 'anneal', 'annotation', 'anns', 'annual', 'anomaly', 'answer', 'ant', 'anti', 'antibody', 'ap', 'application', 'apply', 'approach', 'appropriate', 'approx', 'approximate', 'approximation', 'aptitude', 'arbitrary', 'architecture', 'area', 'argmax', 'argmin', 'argue', 'argument', 'arise', 'arity', 'array', 'arrive', 'arrow', 'art', 'artificial', 'ask', 'asr', 'assess', 'assign', 'assignment', 'associate', 'association', 'assume', 'assumption', 'assure', 'attempt', 'attribute', 'auditory', 'author', 'automate', 'automatic', 'automatically', 'autonomous', '

In [57]:
article_names = []
file = open("nodes.txt", "r")
for line in file:
    article_names.append(line.rstrip("\n"))
file.close()

In [61]:
vectors

<63x1317 sparse matrix of type '<class 'numpy.float64'>'
	with 6404 stored elements in Compressed Sparse Row format>

In [64]:
df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())

In [68]:
df["article_name"] = article_names

In [72]:
#save to a file
df.to_hdf("embeddings.h5", key='df')

In [73]:
#read from file
pd.read_hdf("embeddings.h5", 'df')

,aaai,ab,ability,able,abstract,access,account,accuracy,achieve,acm,...,world,wright,write,year,yield,zealand,zen,zero,zeta,article_name
0,0.000000,0.000000,0.024664,0.000000,0.000000,0.000000,0.000000,0.043163,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Machine_learning
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031660,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Artificial_neural_networks
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Neural_network_software
3,0.000000,0.000000,0.022439,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.022867,0.000000,0.000000,0.000000,0.000000,0.000000,Deep_learning
4,0.000000,0.000000,0.024664,0.000000,0.000000,0.000000,0.000000,0.043163,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Applied_machine_learning
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036335,0.000000,...,0.079808,0.000000,0.000000,0.030695,0.000000,0.000000,0.029068,0.055252,0.000000,AlphaGo
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Unsupervised_learning
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Structured_prediction
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Graphical_models
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Bayesian_networks
